## Requirements

In [ ]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm

## Obtaining a spreadsheet of text which needs to be translated

In [ ]:
squad = load_dataset("squad")

In [ ]:
squad["train"][0]

In [ ]:
text = set()

In [ ]:
def add_to_set(row):
  text.add(row["title"])
  text.add(row["question"])
  text.add(row["context"])
  for ans in row["answers"]["text"]: text.add(ans)

In [ ]:
squad["train"].map(add_to_set)

In [ ]:
df = pd.DataFrame(columns=["en", "mt"])

In [ ]:
for t in tqdm(text): df.loc[len(df.index)] = {"en": t, "mt": ""}

In [ ]:
df.to_excel("translated_train.xlsx")

In [ ]:
f"Train contains {len(df)} pieces of text to translate"

In [ ]:
text = set()

In [ ]:
squad["validation"].map(add_to_set)

In [ ]:
df = pd.DataFrame(columns=["en", "mt"])

In [ ]:
for t in tqdm(text): df.loc[len(df.index)] = {"en": t, "mt": ""}

In [ ]:
df.to_excel("translated_valid.xlsx")

In [ ]:
f"Validation contains {len(df)} pieces of text to translate"

Enter the output spreasheets into Google Sheets and use the ```GOOGLETRANSLATE(text, source language, target language)```

Import the resulting spreadhsheet

## Translating train

In [ ]:
squad = load_dataset("squad", download_mode='force_redownload')

In [ ]:
# translations = pd.read_excel("translated_train.xlsx")
translations = pd.concat([pd.read_excel("translated_train.xlsx"), pd.read_excel("squad_v1_train_errors.xlsx"), pd.read_excel("squad_v1_train_errors (1).xlsx")], ignore_index=True)

In [ ]:
assert len((translations['en'].where(translations['mt'] == '#VALUE!').dropna().values)) == 0

In [ ]:
en_mt_dict = dict()

In [ ]:
for index, row in translations.iterrows():
   en_mt_dict.update({str(row["en"]): str(row["mt"])})

In [ ]:
def translate_row(row):
    row['title'] = en_mt_dict.get(str(row['title']))
    row['context'] = en_mt_dict.get(str(row['context']))
    row['question'] = en_mt_dict.get(str(row['question']))
    row['answers']['text'][0] = en_mt_dict.get(str(row['answers']['text'][0]))
    return row

In [ ]:
squad["train"] = squad["train"].map(translate_row)

In [ ]:
def update_answer_start(row):
    if row['context'] is not None and row['answers']['text'][0] is not None:
        corpus = row['context'].lower()
        substring = row['answers']['text'][0].lower()
        row['answers']['answer_start'][0] = corpus.find(substring)
        return row
    row['answers']['answer_start'][0] = -1
    return row

In [ ]:
squad["train"] = squad["train"].map(update_answer_start)

In [ ]:
squad["train"] = squad["train"].filter(
    lambda row:
    (row["id"] and row["title"] and row["context"] and row["question"] and row['answers']['text'][0] and row['answers']['answer_start'][0])
    and
    (row['question'][-1] == "?")
    and
    (row['answers']['answer_start'][0] != -1)
    and
    (
        row['context'][row['answers']['answer_start'][0] : row['answers']['answer_start'][0] + len(row['answers']['text'][0])].lower()
        ==
        row['answers']['text'][0].lower()
    )
)

In [ ]:
len(squad["train"].filter(lambda row: len(row['answers']['text']) > 0))

In [ ]:
len(squad["train"].filter(lambda row: len(row['answers']['text']) == 0))

In [ ]:
len(squad["train"])

## Errors in train

In [ ]:
translated = squad["train"]

In [ ]:
original = load_dataset("squad", split="train", download_mode='force_redownload')

In [ ]:
all_ids = set()

In [ ]:
def add_to_all(row):
  all_ids.add(row["id"])

In [ ]:
original.map(add_to_all)

In [ ]:
translated_ids = set()

In [ ]:
def add_to_translated(row):
  translated_ids.add(row["id"])

In [ ]:
translated.map(add_to_translated)

In [ ]:
errors = {id for id in all_ids if id not in translated_ids}

In [ ]:
error_entries = original.filter(lambda row: row["id"] in errors)

In [ ]:
len(error_entries)

In [ ]:
error_table = pd.DataFrame(columns=["en", "mt"])

In [ ]:
text = set()

In [ ]:
def add_to_set(row):
  text.add(row["title"])
  text.add(row["question"])
  text.add(row["context"])
  for ans in row["answers"]["text"]: text.add(ans)

In [ ]:
error_entries.map(add_to_set)

In [ ]:
for t in tqdm(text): error_table.loc[len(error_table.index)] = {"en": t, "mt": ""}

In [ ]:
error_table.to_excel("squad_v1_train_errors.xlsx", index=False)

Translate the errors and concatenate to the translations DataFrame. Then run "Translating Train" again

In [ ]:
error_table = pd.read_excel("squad_v1_train_errors.xlsx")

In [ ]:
assert len((error_table['en'].where(error_table['mt'] == '#VALUE!').dropna().values)) == 0

In [ ]:
translations = pd.concat([translations, error_table], ignore_index=True)

In [ ]:
translations.to_excel("translated_train.xlsx", index=False)

## Translating validation

In [ ]:
valid = load_dataset("squad", split="validation", download_mode='force_redownload')

In [ ]:
translations = pd.read_excel("translated_valid.xlsx")

In [ ]:
en_mt_dict = dict()

In [ ]:
for index, row in translations.iterrows():
    en_mt_dict.update({str(row["en"]): str(row["mt"])})

In [ ]:
def translate_row(row):
    row['title'] = en_mt_dict.get(str(row['title']))
    row['context'] = en_mt_dict.get(str(row['context']))
    row['question'] = en_mt_dict.get(str(row['question']))
    for entry in row['answers']['text']: entry = en_mt_dict.get(str(entry))
    return row

In [ ]:
valid = valid.map(translate_row)

In [ ]:
def update_answer_start(row):
    if row['context'] is not None:
        corpus = row['context'].lower()
        starts = []
        answers = []
        for entry in row['answers']['text']:
            search = corpus.find(entry.lower())
            if search != -1:
                answers.append(entry)
                starts.append(search)
        row['answers']['answer_start'] = starts
        row['answers']['text'] = answers
        return row
    return row

In [ ]:
valid = valid.map(update_answer_start)

In [ ]:
def answers_start_match(row):
    matches = []
    for i in range(len(row['answers']['answer_start'])):
        if row['context'][row['answers']['answer_start'][i] : row['answers']['answer_start'][i] + len(row['answers']['text'][i])].lower() == row['answers']['text'][i].lower():
            matches.append(True)
        else:
            matches.append(False)
    return matches

In [ ]:
valid = valid.filter(
    lambda row:
    (row["id"] and row["title"] and row["context"] and row["question"])
    and
    (len(row['answers']['text']) > 0 and len(row['answers']['answer_start']) > 0)
    and
    (row['question'][-1] == "?")
    and
    (all(entry is True for entry in answers_start_match(row)))
)

In [ ]:
len(valid.filter(lambda row: len(row['answers']['text']) > 0))

In [ ]:
len(valid.filter(lambda row: len(row['answers']['text']) == 0))

In [ ]:
squad["validation"] = valid

In [ ]:
len(squad["validation"])

## Errors in validation

In [ ]:
original = load_dataset("squad", split="validation", download_mode='force_redownload')

In [ ]:
all_ids = set()

In [ ]:
def add_to_all(row):
  all_ids.add(row["id"])

In [ ]:
original.map(add_to_all)

In [ ]:
translated_ids = set()

In [ ]:
def add_to_translated(row):
  translated_ids.add(row["id"])

In [ ]:
valid.map(add_to_translated)

In [ ]:
errors = {id for id in all_ids if id not in translated_ids}

In [ ]:
error_entries = original.filter(lambda row: row["id"] in errors)

In [ ]:
len(error_entries)

In [ ]:
error_table = pd.DataFrame(columns=["en", "mt"])

In [ ]:
text = set()

In [ ]:
def add_to_set(row):
  text.add(row["title"])
  text.add(row["question"])
  text.add(row["context"])
  for ans in row["answers"]["text"]: text.add(ans)

In [ ]:
error_entries.map(add_to_set)

In [ ]:
for t in tqdm(text): error_table.loc[len(error_table.index)] = {"en": t, "mt": ""}

In [ ]:
error_table.to_excel("squad_v1_valid_errors.xlsx", index=False)

Translate the errors and concatenate to the translations DataFrame. Then run "Translating Train" again

In [ ]:
error_table = pd.read_excel("squad_v1_valid_errors.xlsx")

In [ ]:
assert len((error_table['en'].where(error_table['mt'] == '#VALUE!').dropna().values)) == 0

In [ ]:
translations = pd.concat([translations, error_table], ignore_index=True)

In [ ]:
translations.to_excel("translated_valid.xlsx", index=False)

## Export dataset

In [ ]:
squad

In [ ]:
squad.save_to_disk("SQuAD_V1")

In [ ]:
# Download as JSON
for split, dataset in squad.items():
    dataset.to_json(f"squad-v1-{split}.json")